In [1]:
import pandas as pd
from estival.wrappers.nevergrad import optimize_model
from tb_incubator.plotting import plot_model_vs_actual, display_plot, plot_tracked_outputs
from tb_incubator.input import load_targets, load_param_info
from tb_incubator.calibrate import get_bcm
from tb_incubator.utils import get_param_table, dict_to_markdown_table
from multiprocessing import cpu_count
pd.options.plotting.backend = "plotly"

In [2]:
fixed_params = load_param_info()["value"]
param_info = load_param_info()
all_targets = load_targets()
covid_effects = {
    'detection_reduction':True
}
acf_screening_rate = {
    2024.0 : 0.0,
    2025.0 : 0.0,
    2026.0 : 0.22,
    2027.0 : 0.22,
    2027.01: 0.0
}
bcm = get_bcm(fixed_params, xpert_improvement=True, covid_effects=covid_effects, acf_screening_rate=acf_screening_rate)

In [3]:
def calibrate_model_with_optimisation(bcm):
    """
    This function performs a model calibration using optimisation. 
    Calibration is performed using the estival package, which implements a wrapper for optimisation methods provided by the nevergrad package. 

    Args:
        bcm: the calibration model object (type BayesianCompartmentalModel) 
    """

    from nevergrad.optimization.differentialevolution import TwoPointsDE
    orunner = optimize_model(bcm, opt_class=TwoPointsDE, num_workers=cpu_count(), budget=4000)
    rec = orunner.minimize(4000)
    optimised_params = rec.value[1]    
   
    return optimised_params, rec

In [4]:
optimised_params = calibrate_model_with_optimisation(bcm)

In [5]:
# run the model with the optimised parameter set
res = bcm.run(optimised_params)
outs = res.get_derived_outputs_df()

AttributeError: 'tuple' object has no attribute 'items'

### Results

In [ ]:
fig = plot_model_vs_actual(
    outs, all_targets['notification'], "notification", "Notification", "Modelled vs Data", "Target data"
)
fig.update_xaxes(range=[1970, 2035])
#display_plot(fig, "genexpert_2points_notif")

In [ ]:
fig = plot_model_vs_actual(
    outs, all_targets["adults_prevalence_pulmonary_target"], "adults_prevalence_pulmonary", "Prevalence (Adult, Pulmonary)", "Modelled vs Data", "Target data"
)
fig.update_xaxes(range=[1970, 2035])